<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/05_Create_Vector_DB/Gemini/Create_Vector_Database_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create Vector Database

In [1]:
#installs
!pip install tqdm
!pip install -q sentence_transformers
!pip install -q pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 5.0 MB/s eta 0:00:00


In [2]:
#imports
from tqdm import tqdm
import os
from google.colab import userdata
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from pinecone import Pinecone, PodSpec, ServerlessSpec
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Import Files From GitHub

In [3]:
# Import github token with google secrets thingy and clone git repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2938, done.
remote: Counting objects: 100% (504/504), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 2938 (delta 286), reused 408 (delta 231), pack-reused 2434
Receiving objects: 100% (2938/2938), 465.48 MiB | 17.88 MiB/s, done.
Resolving deltas: 100% (1918/1918), done.
Updating files: 100% (1209/1209), done.


In [4]:
# Import all cleaned data files
invest_df1 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('/content/Financial_Sentiment_LLM/02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
text_df

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol:\n\nC...",NaN
...,...,...,...,...,...,...,...
10121,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10122,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10123,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10124,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [5]:
# Pull in sentiment analysis results
url =  '/content/Financial_Sentiment_LLM/04_Stock_Modeling/Flat_Stock_Data/Gemini_Prompt2/02_Prompt2_Gemini_Prepped_Stock_Data.csv'
sentiment_df = pd.read_csv(url)

# Pull in stock modeling results
url =  '/content/Financial_Sentiment_LLM/04_Stock_Modeling/Flat_Stock_Data/Gemini_Prompt2/gemini_sentiment_predictions.csv'
prediction_df = pd.read_csv(url)

# Add unique_ID to predictions dataframe
prediction_df['Unique_ID'] = sentiment_df['Unique_ID']
prediction_df

,Source,Ticker,Date,Finance,Production,Reserves/M&A,Regulatory,Green Energy,OG Price,Returns,Predicted_Class,Predicted_Probability_Class_0,Predicted_Probability_Class_1,Unique_ID
0,Investment Research,CVX,2019-01-02,Positive,Positive,Positive,Neutral,Neutral,Neutral,-0.004365,1,0.475544,0.524456,PQ-2463687415
1,Investment Research,CVX,2019-01-02,Positive,Positive,Positive,Neutral,Neutral,Neutral,-0.004365,1,0.475544,0.524456,PQ-2311280728
2,SEC Filings,CVX,2019-01-02,Positive,Neutral,Neutral,Neutral,Neutral,Neutral,-0.004365,0,0.639802,0.360198,IR-4891
3,SEC Filings,XOM,2019-01-02,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0.005200,1,0.471213,0.528787,PQ-2300507429
4,Earnings Call Presentations,XOM,2019-01-02,Positive,Positive,Positive,Neutral,Neutral,Neutral,0.005200,1,0.176905,0.823095,PQ-2311444292
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9469,Investment Research,XOM,2024-06-04,Neutral,Positive,Positive,Neutral,Positive,Neutral,-0.015553,0,0.661627,0.338373,IR-2290
9470,SEC Filings,OXY,2024-06-05,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,-0.000835,0,0.511631,0.488369,IR-2432
9471,Investment Research,EOG,2024-06-05,Positive,Positive,Positive,Neutral,Positive,Neutral,0.004257,1,0.452543,0.547457,PQ-3056009640
9472,ProQuest,DVN,2024-06-05,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,-0.000643,0,0.517298,0.482702,IR-2481


In [8]:
# Merge text and sentiment dataframes
sentiment_df_subset = prediction_df[["Unique_ID","Finance", "Green Energy",
                                    "OG Price", "Reserves/M&A",
                                    "Regulatory", "Production",
                                     "Predicted_Class", "Predicted_Probability_Class_0",
                                     "Predicted_Probability_Class_1"]]

# Drop NANs or articles where sentiment could not be determined
sentiment_df_subset = sentiment_df_subset.dropna()

# Merge the DataFrames on 'Unique_ID'
df_text_sentiment = pd.merge(text_df, sentiment_df_subset, on="Unique_ID", how='right')
df_text_sentiment

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Finance,Green Energy,OG Price,Reserves/M&A,Regulatory,Production,Predicted_Class,Predicted_Probability_Class_0,Predicted_Probability_Class_1
0,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456
1,ProQuest,PQ-2311280728,BP,2019-11-02,"Arabia Saudí valora Aramco entre 1,6 y 1,8 bil...",Arabia Saudí se comprometerá a valorar el giga...,https://www.proquest.com/newspapers/arabia-sau...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456
2,Investment Research,IR-4891,BP,2019-02-20,Reaffirming BUY and $52 price target,"(Arcus\n\nBased in London, BP is one of the wo...",NaN,Positive,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.639802,0.360198
3,ProQuest,PQ-2300507429,BP,2019-10-03,"BP CEO to step down, but no decision on timing",BP PLC boss Bob Dudley is planning to step dow...,https://www.proquest.com/newspapers/bp-ceo-ste...,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,1,0.471213,0.528787
4,ProQuest,PQ-2311444292,BP,2019-11-03,Aramco arranca la OPV para ser la mayor cotiza...,"La compañía estatal saudí Aramco, la mayor pet...",https://www.proquest.com/newspapers/aramco-arr...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.176905,0.823095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10006,Investment Research,IR-2290,XOM,2024-04-30,Exxon Mobil Corporation,"Stock Report | April 30, 2024 | NYSESymbol:\n\...",NaN,Neutral,Positive,Neutral,Positive,Neutral,Positive,0,0.661627,0.338373
10007,Investment Research,IR-2432,XOM,2024-02-05,Exxon Mobil Corporation,"Stock Report Front| February 05, 2024 | NYSESy...",NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.511631,0.488369
10008,ProQuest,PQ-3056009640,XOM,2024-05-17,Why Exxon Stock Can Rise More Plus Netflix Home,"These reports, excerpted and edited by Barron'...",https://www.proquest.com/newspapers/why-exxon-...,Positive,Positive,Neutral,Positive,Neutral,Positive,1,0.452543,0.547457
10009,Investment Research,IR-2481,XOM,2024-01-06,Exxon Mobil Corporation,"Stock Report | January 06, 2024 | NYSE Symbol:...",NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.517298,0.482702


In [9]:
def chunk_text(text):
    """
    Splits a given text into chunks of sentences, each containing up to a specified maximum number of words.

    Parameters:
    text (str): The input text to be split into chunks.

    Returns:
    list: A list of text chunks, where each chunk is a string containing up to the specified maximum number of words.
    """
    try:
      max_word_count = 500
      sentences = sent_tokenize(text)
      chunks = []
      current_chunk = []
      current_word_count = 0

      for sentence in sentences:
          word_count = len(sentence.split())
          if current_word_count + word_count <= max_word_count:
              current_chunk.append(sentence)
              current_word_count += word_count
          else:
              chunks.append(' '.join(current_chunk))
              current_chunk = [sentence]
              current_word_count = word_count

      # Add the last chunk
      if current_chunk:
          chunks.append(' '.join(current_chunk))
    except:
      chunks = ['']

    return chunks

#Write chunks to dataframe
tqdm.pandas()
df_text_sentiment['Text Chunks'] = df_text_sentiment['Article Text'].progress_apply(chunk_text)
df_text_sentiment

100%|██████████| 10011/10011 [00:35<00:00, 280.21it/s]


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Finance,Green Energy,OG Price,Reserves/M&A,Regulatory,Production,Predicted_Class,Predicted_Probability_Class_0,Predicted_Probability_Class_1,Text Chunks
0,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,[If in 70's and 80's the moat of the companies...
1,ProQuest,PQ-2311280728,BP,2019-11-02,"Arabia Saudí valora Aramco entre 1,6 y 1,8 bil...",Arabia Saudí se comprometerá a valorar el giga...,https://www.proquest.com/newspapers/arabia-sau...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,[Arabia Saudí se comprometerá a valorar el gig...
2,Investment Research,IR-4891,BP,2019-02-20,Reaffirming BUY and $52 price target,"(Arcus\n\nBased in London, BP is one of the wo...",NaN,Positive,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.639802,0.360198,"[(Arcus\n\nBased in London, BP is one of the w..."
3,ProQuest,PQ-2300507429,BP,2019-10-03,"BP CEO to step down, but no decision on timing",BP PLC boss Bob Dudley is planning to step dow...,https://www.proquest.com/newspapers/bp-ceo-ste...,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,1,0.471213,0.528787,[BP PLC boss Bob Dudley is planning to step do...
4,ProQuest,PQ-2311444292,BP,2019-11-03,Aramco arranca la OPV para ser la mayor cotiza...,"La compañía estatal saudí Aramco, la mayor pet...",https://www.proquest.com/newspapers/aramco-arr...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.176905,0.823095,"[La compañía estatal saudí Aramco, la mayor pe..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10006,Investment Research,IR-2290,XOM,2024-04-30,Exxon Mobil Corporation,"Stock Report | April 30, 2024 | NYSESymbol:\n\...",NaN,Neutral,Positive,Neutral,Positive,Neutral,Positive,0,0.661627,0.338373,"[Stock Report | April 30, 2024 | NYSESymbol:\n..."
10007,Investment Research,IR-2432,XOM,2024-02-05,Exxon Mobil Corporation,"Stock Report Front| February 05, 2024 | NYSESy...",NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.511631,0.488369,"[Stock Report Front| February 05, 2024 | NYSES..."
10008,ProQuest,PQ-3056009640,XOM,2024-05-17,Why Exxon Stock Can Rise More Plus Netflix Home,"These reports, excerpted and edited by Barron'...",https://www.proquest.com/newspapers/why-exxon-...,Positive,Positive,Neutral,Positive,Neutral,Positive,1,0.452543,0.547457,"[These reports, excerpted and edited by Barron..."
10009,Investment Research,IR-2481,XOM,2024-01-06,Exxon Mobil Corporation,"Stock Report | January 06, 2024 | NYSE Symbol:...",NaN,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,0,0.517298,0.482702,"[Stock Report | January 06, 2024 | NYSE Symbol..."


In [10]:
#split into chunks and create new dataframe
df_chunk = df_text_sentiment.explode('Text Chunks').reset_index(drop=True)
df_chunk['Chunk_ID'] = df_chunk['Unique_ID'] + "-" + df_chunk.index.astype(str)
df_chunk = df_chunk.dropna(subset=['Text Chunks'])
df_chunk

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL,Finance,Green Energy,OG Price,Reserves/M&A,Regulatory,Production,Predicted_Class,Predicted_Probability_Class_0,Predicted_Probability_Class_1,Text Chunks,Chunk_ID
0,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,If in 70's and 80's the moat of the companies ...,PQ-2463687415-0
1,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,"Therefore, for a company to survive beyond its...",PQ-2463687415-1
2,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,RIL has net debt of around Rs 2.7 lakh crore a...,PQ-2463687415-2
3,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,Cloud services enable enterprises to access va...,PQ-2463687415-3
4,ProQuest,PQ-2463687415,BP,2019-09-01,Markets Cover Story: Reliance Great Gamble Wil...,If in 70's and 80's the moat of the companies ...,https://www.proquest.com/newspapers/markets-co...,Positive,Neutral,Neutral,Positive,Neutral,Positive,1,0.475544,0.524456,This new commerce aims to remove inefficiencie...,PQ-2463687415-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65235,Investment Research,IR-2327,XOM,2024-04-06,Exxon Mobil Corporation,"Stock Report | April 06, 2024 | NYSESymbol:\n\...",NaN,Positive,Positive,Negative,Neutral,Neutral,Neutral,1,0.434221,0.565779,"In 2023, liquids production was 2.45 million b...",IR-2327-65235
65236,Investment Research,IR-2327,XOM,2024-04-06,Exxon Mobil Corporation,"Stock Report | April 06, 2024 | NYSESymbol:\n\...",NaN,Positive,Positive,Negative,Neutral,Neutral,Neutral,1,0.434221,0.565779,The unexpected war between Russia and Ukraine ...,IR-2327-65236
65237,Investment Research,IR-2327,XOM,2024-04-06,Exxon Mobil Corporation,"Stock Report | April 06, 2024 | NYSESymbol:\n\...",NaN,Positive,Positive,Negative,Neutral,Neutral,Neutral,1,0.434221,0.565779,Net Margin (%] 10.64 10.95 5.18\n% LT Debt to ...,IR-2327-65237
65238,Investment Research,IR-2327,XOM,2024-04-06,Exxon Mobil Corporation,"Stock Report | April 06, 2024 | NYSESymbol:\n\...",NaN,Positive,Positive,Negative,Neutral,Neutral,Neutral,1,0.434221,0.565779,"By definition,\nintegrated oils operate at bot...",IR-2327-65238


In [11]:
# Create Reference File for getting Chunks

# Determine the number of rows for each split
n = len(df_chunk)
split1 = int(n / 3)
split2 = int(2 * n / 3)

# Split the DataFrame
df1_chunk = df_chunk[:split1]
df2_chunk = df_chunk[split1:split2]
df3_chunk = df_chunk[split2:]

df1_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt1.csv', index=False)
df2_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt2.csv', index=False)
df3_chunk[['Chunk_ID','Text Chunks']].to_csv('/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/Article_Chunk_References_pt3.csv', index=False)

In [12]:

#Commit and Push Changes
!git config --global user.email "bw58@rice.edu"
%cd /content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini/
!git add Article_Chunk_References_pt1.csv
!git commit -m "Add Article_Chunk_References_pt1.csv"
!git add Article_Chunk_References_pt2.csv
!git commit -m "Add Article_Chunk_References_pt2.csv"
!git add Article_Chunk_References_pt3.csv
!git commit -m "Add Article_Chunk_References_pt3.csv"
!git push origin main


/content/Financial_Sentiment_LLM/05_Create_Vector_DB/Gemini
[main 6ba56fa] Add Article_Chunk_References_pt1.csv
 1 file changed, 854365 insertions(+), 838875 deletions(-)
[main d9a7c67] Add Article_Chunk_References_pt2.csv
 1 file changed, 806429 insertions(+), 831796 deletions(-)
[main 1bd32bb] Add Article_Chunk_References_pt3.csv
 1 file changed, 934740 insertions(+), 354597 deletions(-)
Enumerating objects: 21, done.
Counting objects: 100% (21/21), done.
Delta compression using up to 2 threads
Compressing objects: 100% (15/15), done.
Writing objects: 100% (15/15), 24.39 MiB | 1.74 MiB/s, done.
Total 15 (delta 7), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (7/7), completed with 2 local objects.
remote: warning: See https://gh.io/lfs for more information.
remote: warning: File 05_Create_Vector_DB/Gemini/Article_Chunk_References_pt3.csv is 59.00 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB
remote: warning: File 05_Create_Vector_DB/Gem

### Encode Chunks

In [13]:
#create text embeddings of chunks
embedding_model = SentenceTransformer("all-mpnet-base-v2")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
embedding_model.to(device)
text_embeddings = embedding_model.encode(df_chunk['Text Chunks'].tolist())
text_embeddings.shape

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(65240, 768)

### Create Vector Database

In [23]:
#create vector
vectors = []
for idx in range(text_embeddings.shape[0]):
  try:
    vector = [float(i) for i in list(text_embeddings[idx])]
    vectors.append({"id": str(df_chunk['Chunk_ID'][idx]),
                  "metadata": {"date": str(df_chunk['Date'][idx]),
                               "finance": df_chunk['Finance'][idx],
                               "reserves": df_chunk['Reserves/M&A'][idx],
                               "green_energy": df_chunk['Green Energy'][idx],
                               "og_price": df_chunk['OG Price'][idx],
                               "reg": df_chunk['Regulatory'][idx],
                               "prod": df_chunk['Production'][idx],
                               "ticker": df_chunk['Ticker'][idx],
                               "source": df_chunk['Source'][idx],
                               "up_down_prediction": str(df_chunk['Predicted_Class'][idx]),
                               "up_prediction_score": str(df_chunk['Predicted_Probability_Class_1'][idx]),
                               "down_prediction_score": str(df_chunk['Predicted_Probability_Class_0'][idx])},
                    "values": vector })
  except:
    pass



In [25]:
#delete existing index to rebuild with new data
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

pc.delete_index("fastvectors")

In [26]:
#create vector database index
pc = Pinecone(api_key="bc4ea65c-d63e-48e4-9b65-53d6272d927d")

index_name = "fastvectors"

pc.create_index(
    name=index_name,
    dimension=text_embeddings.shape[1], # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [27]:
# insert vectors into database
index = pc.Index(index_name)

# segments set to batch into vector database because of 2MB send limit
segments = 20
sections = int(text_embeddings.shape[0] / segments)
for i in tqdm (range(sections+1), desc="Loading..."):
  index.upsert(
    vectors=vectors[segments*i:segments*(1+i)],
    namespace="ns1"
  )

Loading...: 100%|█████████▉| 3262/3263 [12:18<00:00,  4.41it/s]


PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 22 Jul 2024 04:33:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '74', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1', 'x-pinecone-request-id': '3257119642777207618', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"No vectors provided for upsert request","details":[]}


In [28]:
#check index
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 65460}},
 'total_vector_count': 65460}

In [ ]:
from google.colab import runtime
runtime.unassign()